# Pylabel Prototype
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [5]:
#from io import StringIO
import json
import pandas as pd



## Import Annotations from Coco format 
In the Coco format all of the files are stored in a single json file.

In [20]:
#Download the sample coco file so it can be read and imported
!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/test.json

coco_annnotations = "test.json"

--2021-09-15 14:36:52--  https://raw.githubusercontent.com/pylabelalpha/notebook/main/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322313 (315K) [text/plain]
Saving to: ‘test.json.14’

test.json.14        100%[===================>] 314.76K  --.-KB/s    in 0.03s   

2021-09-15 14:36:52 (9.51 MB/s) - ‘test.json.14’ saved [322313/322313]



In [21]:
def ImportCoco(path):

    with open(path) as cocojson:
        annotations_json = json.load(cocojson)

    #Store the 3 sections of the json as seperate json arrays
    images = pd.json_normalize(annotations_json["images"])
    images = images.rename(columns={"id": "image.id"})
    
    annotations = pd.json_normalize(annotations_json["annotations"])
    categories = pd.json_normalize(annotations_json["categories"])
    categories = categories.rename(columns={"id": "cat.id", "name": "cat.name", "supercategory": "cat.supercategory"})

    df = annotations
    df[['bbox.xmin','bbox.ymax','bbox.width','bbox.height']] = pd.DataFrame(df.bbox.tolist(), index= df.index)
    df = df.drop(['bbox'], axis=1)
    df.insert(8, 'bbox.xmax', df['bbox.xmin'] + df['bbox.width'] )
    df.insert(10, 'bbox.ymin', df['bbox.ymax'] - df['bbox.height'] )

    #Join the annotions with the information about the image to add the image collumns to the dataframe
    df = pd.merge(images, df, left_on='image.id', right_on='image_id', how='left')
    df = pd.merge(df, categories, left_on='category_id', right_on='cat.id', how='left')
    df = df.drop(['image_id','category_id'], axis=1)
    return df

df = ImportCoco(coco_annnotations)

#This dataframe has all of the annotations
df

,file_name,height,width,image.id,area,iscrowd,ignore,segmentation,id,bbox.xmin,bbox.xmax,bbox.ymax,bbox.ymin,bbox.width,bbox.height,cat.supercategory,cat.id,cat.name
0,BloodImage_00007.jpg,480,640,7,37830,0,0,[],1,192,387,91,-103,195,194,none,2,WBC
1,BloodImage_00007.jpg,480,640,7,12390,0,0,[],2,16,134,297,192,118,105,none,1,RBC
2,BloodImage_00007.jpg,480,640,7,12096,0,0,[],3,63,175,371,263,112,108,none,1,RBC
3,BloodImage_00007.jpg,480,640,7,12096,0,0,[],4,118,230,329,221,112,108,none,1,RBC
4,BloodImage_00007.jpg,480,640,7,10010,0,0,[],5,168,259,264,154,91,110,none,1,RBC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,BloodImage_00407.jpg,480,640,407,10368,0,0,[],941,510,606,93,-15,96,108,none,1,RBC
941,BloodImage_00407.jpg,480,640,407,31265,0,0,[],942,471,640,289,104,169,185,none,2,WBC
942,BloodImage_00407.jpg,480,640,407,5180,0,0,[],943,412,482,229,155,70,74,none,3,Platelets
943,BloodImage_00407.jpg,480,640,407,2070,0,0,[],944,67,113,112,67,46,45,none,3,Platelets


In [54]:

df_left

,file_name,height,width,id_x,area,iscrowd,category_id,ignore,segmentation,image_id,id_y,bbox.xmin,xmax,bbox.ymax,ymin,bbox.width,bbox.height
0,BloodImage_00007.jpg,480,640,7,37830,0,2,0,[],7,1,192,387,91,-103,195,194
1,BloodImage_00007.jpg,480,640,7,12390,0,1,0,[],7,2,16,134,297,192,118,105
2,BloodImage_00007.jpg,480,640,7,12096,0,1,0,[],7,3,63,175,371,263,112,108
3,BloodImage_00007.jpg,480,640,7,12096,0,1,0,[],7,4,118,230,329,221,112,108
4,BloodImage_00007.jpg,480,640,7,10010,0,1,0,[],7,5,168,259,264,154,91,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,BloodImage_00407.jpg,480,640,407,10368,0,1,0,[],407,941,510,606,93,-15,96,108
941,BloodImage_00407.jpg,480,640,407,31265,0,2,0,[],407,942,471,640,289,104,169,185
942,BloodImage_00407.jpg,480,640,407,5180,0,3,0,[],407,943,412,482,229,155,70,74
943,BloodImage_00407.jpg,480,640,407,2070,0,3,0,[],407,944,67,113,112,67,46,45


In [ ]:
#convert annotations to a dataframe 
df = pd.json_normalize(categories)
df[['bbox.xmin','bbox.ymax','bbox.width','bbox.height']] = pd.DataFrame(df.bbox.tolist(), index= df.index)
df = df.drop(['bbox'], axis=1)
df.insert(8, 'xmax', df['bbox.xmin'] + df['bbox.width'] )
df.insert(10, 'ymin', df['bbox.ymax'] - df['bbox.height'] )

df_left = pd.merge(images, images, left_on='id', right_on='image_id', how='left')
df_left